# Predicting FWI using deepFWI Pre-trained models

### Introduction
The Fire Weather Index (FWI) is a meteorologically based index used worldwide to estimate fire danger. It consists of different components that account for the effects of fuel moisture and wind on fire behaviour and spread. The higher the FWI, the more favorable the meteorological conditions to trigger a wildfire. This indicator can help shape long-term tourist strategy and to plan future investments under a changing climate.

The [`esowc/wildfire-forecasting`](https://github.com/esowc/wildfire-forecasting) project intends to reproduce the Fire Forecasting capabilities of GEFF using Deep Learning and develop further improvements in accuracy, geography and time scale through inclusion of additional variables or optimisation of model architecture & hyperparameters. 

In this notebook we demonstrate the use of pre-trained Deep Learning models to perform FWI inference. The Deep Learning Model has been trained on 1 year of data and takes as input the four weather forcings used in the numerical calculation of FWI:
```
* t2     -    Temperature at 2m (K)
* tp     -    Total Precipitation accumulated over the previous 24 hours (mm)
* rh     -    Relative Humidity (%)
* wspeed -    Windspeed (m/s)
```

These input weather forcings are supplied for 7 days and the model produces FWI forecast for 21 days. There is 1 day of overlap between the 4 weather forcing variable inputs and the output FWI forecast as indicated below:
```
                                       ____________
Weather Forcings (t-6, t-5, ..., t) -> | deepFWI  |  -> deepFWI-Forecast (t, t+1, t+2, ..., t+20)
                                       ------------
```

To run this notebook, make sure you have the `wildfire-dl` `conda` environment activated. Instructions for setting up the environment are available in the project [`README`](https://github.com/esowc/wildfire-forecasting/blob/master/README.md).

In [1]:
import os
from IPython.display import Image
os.chdir('..')

### Ensure access to the data

This example will require the following dataset from gs://deepgeff-data-v0/ -
* `fwi-forcings/`: Weather Forcings for 2020-12-01 to 2020-12-07 --> 7 days --> 7 files 
* `fwi-reanalysis/`: ERA Reanalysis of FWI for 2020-12-07 to 2020-12-27 --> 21 days --> 21 files

If you wish to explore these datasets further, have a look at the EDA notebooks in `data/EDA/` in the repository.

In [2]:
# Uncomment to download a minimal sample dataset consisting of Weather Forcings
# for 2019-12-01 to 2019-12-07 --> 7 days --> 7 files
!mkdir sampledata/7_21/forcings -p
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/forcings/ECMWF_FO_2019120[1-7]*' sampledata/7_21/forcings

In [3]:
# Uncomment to download a minimal sample dataset consisting of ERA Reanalysis 
# for 2019-12-07 to 2019-12-27 --> 21 days --> 21 files
!mkdir sampledata/7_21/fwi-reanalysis -p
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/fwi-reanalysis/ECMWF_FWI_201912[0][7-9]*' sampledata/7_21/fwi-reanalysis
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/fwi-reanalysis/ECMWF_FWI_201912[1][0-9]*' sampledata/7_21/fwi-reanalysis
!gsutil -m -q cp -r 'gs://deepgeff-data-v0/fwi-reanalysis/ECMWF_FWI_201912[2][0-7]*' sampledata/7_21/fwi-reanalysis

### Getting the pre-trained model

For this example notebook, we use the following model-

- Run ID: `2tcenlrp` that has been trained for 30 epochs on 4 months of T2, TP, RH and WSpeed data for April 2019 - Sept, 2019.

If you wish to explore other models, have a look at pre-trained_models.md.

In [4]:
# Uncomment to download a sample pre-trained model checkpoint file
!mkdir samplemodel/7_21/ -p
!gsutil -q cp 'gs://deepgeff-models-v0/pre-trained_models/7_21/2tcenlrp/2tcenlrp_29_30.ckpt' samplemodel/7_21/

### Running the inference

For performing inference and then benchmarking the results, we use [`src/test.py`](https://github.com/esowc/wildfire-forecasting/blob/master/src/test.py) which takes as argument a dict of configuration parameters. 

Providing the configuration options,

```
* in_days        -    no of input days of weather forcings
* out_days       -    no of output days of fwi prediction
* checkpoint_file-    path to pretrained checkpoint file
* reanalysis_dir -    path to fwi-reanalysis data
* forcings_dir   -    path to fwi-forcings data
```

In [5]:
!python src/test.py -in-days 7 -out-days 21 -checkpoint-file "samplemodel/7_21/2tcenlrp_29_30.ckpt" \
-forcings-dir "sampledata/7_21/forcings" -reanalysis-dir "sampledata/7_21/fwi-reanalysis"

 |       init-features -> 16                  
 |             in-days -> 7                   
 |            out-days -> 21                  
 |              epochs -> 100                 
 |       learning-rate -> 0.001               
 |          batch-size -> 1                   
 |               split -> 0.2                 
 |           use-16bit -> True                
 |                gpus -> 1                   
 |               optim -> one_cycle           
 |             dry-run -> False               
 |             find-lr -> False               
 |           search-bs -> False               
 |          case-study -> False               
 |         clip-output -> False               
 |              boxcox -> 0.1182              
 |              binned -> [0, 5.2, 11.2, 21.3, 38.0, 50]
 |       round-to-zero -> False               
 |         isolate-frp -> False               
 |          date-range -> False               
 |             cb-loss -> False               
 | 

 'mae_11.2_21.3_19': tensor(10.5064, device='cuda:0'),
 'mae_11.2_21.3_2': tensor(9.3386, device='cuda:0'),
 'mae_11.2_21.3_20': tensor(10.0396, device='cuda:0'),
 'mae_11.2_21.3_3': tensor(9.2536, device='cuda:0'),
 'mae_11.2_21.3_4': tensor(9.2497, device='cuda:0'),
 'mae_11.2_21.3_5': tensor(9.5806, device='cuda:0'),
 'mae_11.2_21.3_6': tensor(9.5250, device='cuda:0'),
 'mae_11.2_21.3_7': tensor(10.0858, device='cuda:0'),
 'mae_11.2_21.3_8': tensor(10.3736, device='cuda:0'),
 'mae_11.2_21.3_9': tensor(10.5553, device='cuda:0'),
 'mae_12': tensor(5.9076, device='cuda:0'),
 'mae_13': tensor(15.2627, device='cuda:0'),
 'mae_14': tensor(6.1333, device='cuda:0'),
 'mae_15': tensor(6.0445, device='cuda:0'),
 'mae_16': tensor(6.3538, device='cuda:0'),
 'mae_17': tensor(6.8552, device='cuda:0'),
 'mae_18': tensor(7.0286, device='cuda:0'),
 'mae_19': tensor(17.8161, device='cuda:0'),
 'mae_2': tensor(20.1055, device='cuda:0'),
 'mae_20': tensor(7.4938, device='cuda:0'),
 'mae_21.3_38.0_0': t

 'mse_21.3_38.0_20': tensor(339.0386, device='cuda:0'),
 'mse_21.3_38.0_3': tensor(227.7437, device='cuda:0'),
 'mse_21.3_38.0_4': tensor(233.2883, device='cuda:0'),
 'mse_21.3_38.0_5': tensor(273.1041, device='cuda:0'),
 'mse_21.3_38.0_6': tensor(297.3940, device='cuda:0'),
 'mse_21.3_38.0_7': tensor(264.9616, device='cuda:0'),
 'mse_21.3_38.0_8': tensor(288.3423, device='cuda:0'),
 'mse_21.3_38.0_9': tensor(310.7574, device='cuda:0'),
 'mse_3': tensor(533.9545, device='cuda:0'),
 'mse_38.0_50_0': tensor(205.9284, device='cuda:0'),
 'mse_38.0_50_1': tensor(287.4853, device='cuda:0'),
 'mse_38.0_50_10': tensor(468.9977, device='cuda:0'),
 'mse_38.0_50_11': tensor(493.7505, device='cuda:0'),
 'mse_38.0_50_12': tensor(506.1502, device='cuda:0'),
 'mse_38.0_50_13': tensor(502.0375, device='cuda:0'),
 'mse_38.0_50_14': tensor(418.5078, device='cuda:0'),
 'mse_38.0_50_15': tensor(458.9712, device='cuda:0'),
 'mse_38.0_50_16': tensor(524.3126, device='cuda:0'),
 'mse_38.0_50_17': tensor(634.

### Results

We demonstrate below the metrics (accuracy, mean-squared-error and mean-absolute-error) for deepFWI predicted values. These metrics are for 1 FWI prediction (for 14 days) in a global setting. The metrics are calculated against fwi-reanalysis (considered the ground truth). The grouped bar plots indicate the metrics for deepFWI while the line plots are used for fwi-forecast. The FWI values are binned as per the EFFIS categorisation:

```
* [0, 5.2]     - Very Low
* (5.2, 11.2]  - Low
* (11.2, 21.3] - Moderate
* (21.3, 38.0] - High
* (38.0, 50.0] - Very High
* (50.0, Inf)  - Extreme
```

As seen, the deepFWI predictions are consistent with FWI-Forecast and even offer better accuracy for longer term predictions in certain FWI categories. 

In [6]:
# Generate result plots
!python src/plot.py -f result.json -i 7 -o 21
!ls acc* mae* mse*

acc_20201213-062406.png  mae_20201213-062416.png  mse_20201213-062411.png


In [7]:
Image(filename='acc_20201213-062406.png')